<a href="https://colab.research.google.com/github/harshitaachadha/Data-Mining-Script-Rating-Prediction/blob/main/Named_entity_recognition_using_spacy_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To do list -
- [X] Number of different locations
- [ ] Emotional Archetypes
- [X] lines per character
- [ ] frequent character sets
- [X] Total spoken words
- [X] list of characters


In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=41b9717f6aa9d0ab2a9eb46dbbdbef554e66b1b71123364454beb2887239ace3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
import spacy
import en_core_web_sm
import re
nlp = en_core_web_sm.load()

In [22]:
import pandas as pd
df = pd.read_csv('ready_for_mining_script.csv')
df.head()

,Unnamed: 0,text script,Title,Episode,Season
0,0,"[Scene: The Subway, Phoebe is singing for\ncha...",The Pilot-The Uncut Version,1.0,1.0
1,1,"[Scene Central Perk, everyone's there.]\nMonic...",The One With The Sonogram At the End,2.0,1.0
2,2,"[Scene: Chandler and Joey's, Chandler is helpi...",The One With The Thumb,3.0,1.0
3,3,"[Scene: Central Perk, Ross and Monica are watc...",The One With George Stephanopoulos,4.0,1.0
4,4,"[Scene: Central Perk, all six are there.]\nMon...",The One With The East German Laundry Detergant,5.0,1.0


In [24]:
def extract_character_names(text):
    doc = nlp(text)
    pattern = r'(\w+\s*):'
    matches = re.findall(pattern, text)
    filtered_matches = [match for match in matches if any(ent.text == match for ent in doc.ents)]
    unique_character_names = list(set(filtered_matches))
    return unique_character_names

In [25]:
from tqdm import tqdm
tqdm.pandas()
df['character_names'] = df['text script'].progress_apply(extract_character_names)

100%|██████████| 229/229 [04:14<00:00,  1.11s/it]


In [26]:
df.shape

(229, 6)

In [28]:
df.tail()

,Unnamed: 0,text script,Title,Episode,Season,character_names
224,224,"[Scene: Central Perk. Phoebe, Monica and Chand...",The One With Princess Consuela,14.0,10.0,"[Mark, Joey, Rachel, Clerk, Chandler, Campbell..."
225,225,[Scene: Central Perk. Phoebe's reading a newsp...,The One Where E...,15.0,10.0,"[Joey, Rachel, Realtor, Chandler, Ross, Monica..."
226,226,[Scene: Joey's place. Rachel and Joey \n ...,The One With Ra...,16.0,10.0,"[Rachel, Joey, Chandler, Erica, Ross, Monica, ..."
227,227,[Scene: Monica and Chandler's apartment. It's ...,The Last One,17.0,10.0,"[Rachel, Joey, 2, Chandler, Erica, Ross, Gunth..."
228,228,[Scene: In a TV commercial that the gang is wa...,The One After the Superbowl (2),13.0,2.0,"[JANITOR, ROB]"


In [29]:
#so now we have a character list for each episode that is non-empty. I will still not do any text cleaning becuase episodes have different ways in which character
#lines are represented instead i'll first get the words per character per episode.
def count_words(text):
    words_spoken = {}
    last_character = None
    lines = text.split('\n')
    for line in lines:
        # Remove text within square brackets and parentheses
        line = re.sub(r'\[.*?\]|\(.*?\)', '', line)
        if ':' in line:
            character, spoken_text = line.split(':', 1)
            character = character.strip()
            spoken_text = spoken_text.strip()
            if character not in words_spoken:
                words_spoken[character] = 0
            words = spoken_text.split()
            words_spoken[character] += len(words)
            last_character = character
        elif last_character is not None:
            # Append text to the last character's count
            spoken_text = line.strip()
            words = spoken_text.split()
            words_spoken[last_character] += len(words)
    return words_spoken

tqdm.pandas()
df['Spoken Word List'] = df['text script'].progress_apply(count_words)

100%|██████████| 229/229 [00:00<00:00, 452.45it/s]


In [33]:
df.head()

,Unnamed: 0,text script,Title,Episode,Season,character_names,Spoken Word List
0,0,"[Scene: The Subway, Phoebe is singing for\ncha...",The Pilot-The Uncut Version,1.0,1.0,"[Joey, Paul, Rachel, Frannie, Chandler, Ross, ...","{'[Scene': 81, 'Phoebe': 166, 'Ross': 357, 'Jo..."
1,1,"[Scene Central Perk, everyone's there.]\nMonic...",The One With The Sonogram At the End,2.0,1.0,"[Rachel, Joey, Barry, Scene, Geller, Chandler,...","{'Monica': 253, 'Joey': 73, 'Phoebe': 115, 'Ra..."
2,2,"[Scene: Chandler and Joey's, Chandler is helpi...",The One With The Thumb,3.0,1.0,"[Joey, Rachel, Scene, Chandler, Lizzie, Ross, ...","{'Chandler': 449, 'Joey': 185, 'Monica': 480, ..."
3,3,"[Scene: Central Perk, Ross and Monica are watc...",The One With George Stephanopoulos,4.0,1.0,"[Joey, Rachel, Receptionist, Chandler, Ross, J...","{'[Scene': 158, 'Monica': 418, 'Ross': 440, 'P..."
4,4,"[Scene: Central Perk, all six are there.]\nMon...",The One With The East German Laundry Detergant,5.0,1.0,"[Angela, Joey, Rachel, Bob, Chandler, Ross, Mo...","{'Monica': 323, 'Ross': 560, 'Rachel': 371, 'C..."


In [35]:
def calculate_total_words(words_spoken):
    total = 0
    for character, word_count in words_spoken.items():
        total += word_count
    return total

tqdm.pandas()
df['Total Spoken Words'] = df['Spoken Word List'].progress_apply(calculate_total_words)

100%|██████████| 229/229 [00:00<00:00, 74439.71it/s]


In [37]:
df.head()

,Unnamed: 0,text script,Title,Episode,Season,character_names,Spoken Word List,Total Spoken Words
0,0,"[Scene: The Subway, Phoebe is singing for\ncha...",The Pilot-The Uncut Version,1.0,1.0,"[Joey, Paul, Rachel, Frannie, Chandler, Ross, ...","{'[Scene': 81, 'Phoebe': 166, 'Ross': 357, 'Jo...",2290
1,1,"[Scene Central Perk, everyone's there.]\nMonic...",The One With The Sonogram At the End,2.0,1.0,"[Rachel, Joey, Barry, Scene, Geller, Chandler,...","{'Monica': 253, 'Joey': 73, 'Phoebe': 115, 'Ra...",2520
2,2,"[Scene: Chandler and Joey's, Chandler is helpi...",The One With The Thumb,3.0,1.0,"[Joey, Rachel, Scene, Chandler, Lizzie, Ross, ...","{'Chandler': 449, 'Joey': 185, 'Monica': 480, ...",2336
3,3,"[Scene: Central Perk, Ross and Monica are watc...",The One With George Stephanopoulos,4.0,1.0,"[Joey, Rachel, Receptionist, Chandler, Ross, J...","{'[Scene': 158, 'Monica': 418, 'Ross': 440, 'P...",2721
4,4,"[Scene: Central Perk, all six are there.]\nMon...",The One With The East German Laundry Detergant,5.0,1.0,"[Angela, Joey, Rachel, Bob, Chandler, Ross, Mo...","{'Monica': 323, 'Ross': 560, 'Rachel': 371, 'C...",2765


In [41]:
import nltk
nltk.download('punkt')
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
def extract_locations_organizations(text):
    PATH_TO_JAR = 'stanford-ner.jar'
    PATH_TO_MODEL = 'english.all.3class.distsim.crf.ser.gz'
    tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL, path_to_jar=PATH_TO_JAR, encoding='utf-8')
    bracket_contents = re.findall(r'\[(.*?)\]', text)
    locations = []
    organizations = []

    for bracket_text in bracket_contents:
        words = word_tokenize(bracket_text)
        ner_tags = tagger.tag(words)

        current_location = []
        current_organization = []

        for word, tag in ner_tags:
            if tag == 'LOCATION':
                current_location.append(word)
            elif tag == 'ORGANIZATION':
                current_organization.append(word)

        if current_location:
            locations.append(" ".join(current_location))
        if current_organization:
            organizations.append(" ".join(current_organization))

    return locations + organizations

In [ ]:
tqdm.pandas()
df['List of Locations'] = df['text script'].progress_apply(extract_locations_organizations)

In [44]:
df.head()

,Unnamed: 0,text script,Title,Episode,Season,character_names,Spoken Word List,Total Spoken Words,List of Locations
0,0,"[Scene: The Subway, Phoebe is singing for\ncha...",The Pilot-The Uncut Version,1.0,1.0,"[Joey, Paul, Rachel, Frannie, Chandler, Ross, ...","{'[Scene': 81, 'Phoebe': 166, 'Ross': 357, 'Jo...",2290,[Iridium]
1,1,"[Scene Central Perk, everyone's there.]\nMonic...",The One With The Sonogram At the End,2.0,1.0,"[Rachel, Joey, Barry, Scene, Geller, Chandler,...","{'Monica': 253, 'Joey': 73, 'Phoebe': 115, 'Ra...",2520,[Central Park]
2,2,"[Scene: Chandler and Joey's, Chandler is helpi...",The One With The Thumb,3.0,1.0,"[Joey, Rachel, Scene, Chandler, Lizzie, Ross, ...","{'Chandler': 449, 'Joey': 185, 'Monica': 480, ...",2336,"[Central Perk, Iridium, Iridium, Cental Perk]"
3,3,"[Scene: Central Perk, Ross and Monica are watc...",The One With George Stephanopoulos,4.0,1.0,"[Joey, Rachel, Receptionist, Chandler, Ross, J...","{'[Scene': 158, 'Monica': 418, 'Ross': 440, 'P...",2721,[]
4,4,"[Scene: Central Perk, all six are there.]\nMon...",The One With The East German Laundry Detergant,5.0,1.0,"[Angela, Joey, Rachel, Bob, Chandler, Ross, Mo...","{'Monica': 323, 'Ross': 560, 'Rachel': 371, 'C...",2765,[]


In [45]:
df.to_csv('the_one_with_the_NER.csv')